In [1]:
import sys
import os

current_directory = os.getcwd()
path = os.path.dirname(current_directory)
sys.path.append(path)
from Utils import *

%matplotlib widget
from ipywidgets import interact, interactive, widgets
from matplotlib.patches import Rectangle, Circle, Arrow

In [2]:
import glob
import os

import tempfile
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Any, Mapping, Hashable


# Load images
srcdir = os.getcwd()
# Create dictionary for MONAI
imageNumber = 3 # train_files[0] = 2D images COR  / train_files[1] = 2D images SAG / train_files[2] = 3D images COR

if imageNumber==1:
    orientation = 'LIP'
else:
    orientation = 'PIL'
    
print('Reading images from: ' + srcdir)
images_1 = sorted(glob.glob(os.path.join(srcdir, 'test_dataset', '*_M.nii.gz')))
images_2 = sorted(glob.glob(os.path.join(srcdir, 'test_dataset', '*_P.nii.gz')))

print(images_1[imageNumber])
# Show original image
sitk_image_1 = sitk.ReadImage(images_1[imageNumber], sitk.sitkFloat32)
sitk_image_2 = sitk.ReadImage(images_2[imageNumber], sitk.sitkFloat32)
show_mag_phase_images(sitk_image_1, sitk_image_2, 'Selected Sitk image pair')


## Crop image to have an asymmetrical volume to make sure dimensions are in correct order
## Comment out after saved cropped version of the images to be used
# sitk_image_1 = cropItk(sitk_image_1_orig, 56, side='top')
# sitk_image_2 = cropItk(sitk_image_2_orig, 56, side='top')
# print(sitk_image_1.GetSize())
# show_mag_phase_images(sitk_image_1, sitk_image_2, 'Cropped sitk image from nii')
# filename_1 = images_1[imageNumber].split('_M.nii.gz')[0] + '_crop_M.nii.gz'
# filename_2 = images_2[imageNumber].split('_P.nii.gz')[0] + '_crop_P.nii.gz'
# sitk.WriteImage(sitk_image_1, filename_1)
# sitk.WriteImage(sitk_image_2, filename_2)

Reading images from: /Users/pl771/Devel/MRINeedleSegmentation/TestingNotebook
/Users/pl771/Devel/MRINeedleSegmentation/TestingNotebook/test_dataset/z000_RealImage_2D_COR_M.nii.gz


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

In [3]:
import monai
from monai.config import print_config
from monai.utils import first
from monai.config import KeysCollection
from monai.data import Dataset, ArrayDataset, create_test_image_3d, DataLoader
from monai.transforms import (
    Transform,
    Compose,
    EnsureChannelFirstd,
    LoadImaged,
    ScaleIntensityd,
    Orientationd,
)
from sitkIO import PushSitkImaged

print('Sitk image object')
print(sitk_image_1.GetSize())

# Create dictionary
train_files = [
    {'image_1': image_1_name, 'image_2': image_2_name}
    for image_1_name, image_2_name in zip(images_1, images_2)
]
print(train_files)
img_1_file = train_files[imageNumber]['image_1']
img_2_file = train_files[imageNumber]['image_2']
fn_keys = ['image_1', 'image_2']  # filename keys for image files

LoadTransf = Compose([
                        LoadImaged(keys=fn_keys, image_only=True),
                        EnsureChannelFirstd(keys=fn_keys, channel_dim='no_channel'),
                    ])
tensor_images = LoadTransf(train_files[imageNumber])
print('Load')
print(tensor_images['image_1'].shape)

orientTransf = Compose([
                        Orientationd(keys=fn_keys, axcodes=orientation)])
tensor_images = orientTransf(tensor_images)
print('Orientation')
print(tensor_images['image_1'].shape)

# Check pixel order (L-R, P-A, I-S)
w, h, d = 91, 29, 0 # Change to different values
sitk_array = sitk.GetArrayFromImage(sitk_image_1) # This converts to (D, H, W)
metatensor_array = tensor_images['image_1'].as_tensor().numpy()[0] # Get first channel - Metatensor is already in (D, H, W)

print(sitk_image_1[w,h,d])
print(sitk_array[d, h, w])
print(metatensor_array[d, h, w])

# Put back do sitk
sitkTransform = Compose([PushSitkImaged(keys=fn_keys, resample=True, output_dtype=np.float32, print_log=False)])
sitk_output = sitkTransform(tensor_images)
show_mag_phase_images(sitk_output['image_1'], sitk_output['image_2'], title= 'Image pair after PushSitk')

Sitk image object
(256, 256, 1)
[{'image_1': '/Users/pl771/Devel/MRINeedleSegmentation/TestingNotebook/test_dataset/000_RealImage_2D_COR_crop_M.nii.gz', 'image_2': '/Users/pl771/Devel/MRINeedleSegmentation/TestingNotebook/test_dataset/000_RealImage_2D_COR_crop_P.nii.gz'}, {'image_1': '/Users/pl771/Devel/MRINeedleSegmentation/TestingNotebook/test_dataset/001_RealImage_2D_SAG_crop_M.nii.gz', 'image_2': '/Users/pl771/Devel/MRINeedleSegmentation/TestingNotebook/test_dataset/001_RealImage_2D_SAG_crop_P.nii.gz'}, {'image_1': '/Users/pl771/Devel/MRINeedleSegmentation/TestingNotebook/test_dataset/002_SyntheticImage_3D_COR_crop_M.nii.gz', 'image_2': '/Users/pl771/Devel/MRINeedleSegmentation/TestingNotebook/test_dataset/002_SyntheticImage_3D_COR_crop_P.nii.gz'}, {'image_1': '/Users/pl771/Devel/MRINeedleSegmentation/TestingNotebook/test_dataset/z000_RealImage_2D_COR_M.nii.gz', 'image_2': '/Users/pl771/Devel/MRINeedleSegmentation/TestingNotebook/test_dataset/z000_RealImage_2D_COR_P.nii.gz'}, {'ima

RuntimeError: applying transform <sitkIO.PushSitkImaged object at 0x11c5e0eb0>